<a href="https://colab.research.google.com/github/SobhanSah00/Credit_Score_Classifiear/blob/main/Credit_Score_Classifiear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the necessary libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [2]:
# import the csv file
data = pd.read_csv('train.csv')

# show the data after changes
data.head()

<ipython-input-2-18baf3cf2605>:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('train.csv')


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [3]:
''' Handel The Missing Value '''

# Replace the Missing Value with Median
data['Monthly_Inhand_Salary'].fillna(data['Monthly_Inhand_Salary'].median(), inplace=True)

# Replace the TypeOfLoan with Most Frequent Values(:It is mostly Catagorical)
data['Type_of_Loan'].fillna(data['Type_of_Loan'].mode()[0], inplace=True)

# Fixed Inconsistent Values

# Convert to numbers, set invalid entries to NaN
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

# Fille up NaNs with median
data['Age'].fillna(data['Age'].median(), inplace=True)

# standardaized the Credit History Age For better known (:like converting the all in to months)
def AllConvertingToMongths(age_str):
  # Handle missing values
    if pd.isnull(age_str):
        return None
    years, months = 0, 0

    # This is devide the age_str to parts and stores in array name as parts
    parts = age_str.split()

    # Extract years
    if "Year" in parts:
        years = int(parts[0])

    # Extract months if present
    if "Month" in parts:
        months_index = parts.index("Month") - 1
        months = int(parts[months_index])

    return years * 12 + months


data['Credit_History_Months'] = data['Credit_History_Age'].apply(AllConvertingToMongths)

<ipython-input-3-19090e4c15df>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Monthly_Inhand_Salary'].fillna(data['Monthly_Inhand_Salary'].median(), inplace=True)
<ipython-input-3-19090e4c15df>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col

In [4]:
''' for now please run it once , if u do then it shoing error '''

# Convert texutual information

''' we can also do like this '''

# data['Occupation'] = data['Occupation'].astype('category')
# data['Credit_Mix'] = data['Credit_Mix'].astype('category')

''' it create copy of uniqe features and then replace with it(:why we do this because machine does not understand the textual information so we convert to some type oridinal value )
 (here we use get_dummies for create copy on column and use one_hot_coading concept)  '''


''' why i write this function ?
      -> like before when the single like is written then when i run this code 1st time then the code well and fine not having any problem , if bichance i change my code and re run again then this code
      snippet is shwos me error , why ? because when we create dummes by the help pd.get_dummies and then replace with original column then their the name also change in dataset for permanent so if there is not
      this type of feature is present then ovious it showing us the error .

      data = pd.get_dummies(data, columns=['Occupation', 'Credit_Mix'], drop_first=True)
'''

# def apply_only_ones_hot_encoding(data, columns_to_encode):
#     # Drop existing one-hot encoded columns before applying pd.get_dummies
#     data = data.drop(columns=[col for col in data.columns if col.startswith(tuple(columns_to_encode))], axis=1, errors='ignore')

#     # display(columns_to_encode)

#     # Apply one-hot encoding to the specified columns
#     data = pd.get_dummies(data, columns=columns_to_encode, drop_first=True)

#     return data

# data = apply_only_ones_hot_encoding(data, ['Occupation','Credit_Mix'])

data = pd.get_dummies(data, columns=['Occupation', 'Credit_Mix'], drop_first=True)
display(data)

,ID,Customer_ID,Month,Name,Age,SSN,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,...,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Occupation________,Credit_Mix_Good,Credit_Mix_Standard,Credit_Mix__
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.0,821-00-0265,19114.12,1824.843333,3,4,...,False,False,False,True,False,False,False,False,False,True
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23.0,821-00-0265,19114.12,3093.745000,3,4,...,False,False,False,True,False,False,False,True,False,False
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500.0,821-00-0265,19114.12,3093.745000,3,4,...,False,False,False,True,False,False,False,True,False,False
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23.0,821-00-0265,19114.12,3093.745000,3,4,...,False,False,False,True,False,False,False,True,False,False
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23.0,821-00-0265,19114.12,1824.843333,3,4,...,False,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25.0,078-73-5990,39628.99,3359.415833,4,6,...,True,False,False,False,False,False,False,False,False,True
99996,0x25fea,CUS_0x942c,May,Nicks,25.0,078-73-5990,39628.99,3359.415833,4,6,...,True,False,False,False,False,False,False,False,False,True
99997,0x25feb,CUS_0x942c,June,Nicks,25.0,078-73-5990,39628.99,3359.415833,4,6,...,True,False,False,False,False,False,False,True,False,False
99998,0x25fec,CUS_0x942c,July,Nicks,25.0,078-73-5990,39628.99,3359.415833,4,6,...,True,False,False,False,False,False,False,True,False,False


In [5]:
data.head()

,ID,Customer_ID,Month,Name,Age,SSN,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,...,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Occupation________,Credit_Mix_Good,Credit_Mix_Standard,Credit_Mix__
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.0,821-00-0265,19114.12,1824.843333,3,4,...,False,False,False,True,False,False,False,False,False,True
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23.0,821-00-0265,19114.12,3093.745000,3,4,...,False,False,False,True,False,False,False,True,False,False
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500.0,821-00-0265,19114.12,3093.745000,3,4,...,False,False,False,True,False,False,False,True,False,False
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23.0,821-00-0265,19114.12,3093.745000,3,4,...,False,False,False,True,False,False,False,True,False,False
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23.0,821-00-0265,19114.12,1824.843333,3,4,...,False,False,False,True,False,False,False,True,False,False


In [6]:
# lebel encoading for ordinal datas

'''
Label Encoding is a technique used to convert categorical variables into numeric form.
This method is particularly useful for ordinal categorical variables,
where the categories have a meaningful order or ranking.
'''
score_mapping = {'Poor': 0, 'Standard': 1, 'Good': 2}
data['Credit_Score'] = data['Credit_Score'].map(score_mapping)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 45 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  float64
 5   SSN                       100000 non-null  object 
 6   Annual_Income             100000 non-null  object 
 7   Monthly_Inhand_Salary     100000 non-null  float64
 8   Num_Bank_Accounts         100000 non-null  int64  
 9   Num_Credit_Card           100000 non-null  int64  
 10  Interest_Rate             100000 non-null  int64  
 11  Num_of_Loan               100000 non-null  object 
 12  Type_of_Loan              100000 non-null  object 
 13  Delay_from_due_date       100000 non-null  in

In [8]:
data['Credit_Score'].unique()

array([2, 1, 0])

In [9]:
# Standardized the Numerical colmuns or data

from sklearn.preprocessing import StandardScaler

# Remove non-numeric characters (like underscores, $, etc.) and convert to float
data['Monthly_Inhand_Salary'] = data['Monthly_Inhand_Salary'].replace(r'[^0-9.]', '', regex=True).astype(float)
data['Num_Bank_Accounts'] = data['Num_Bank_Accounts'].astype(int)  # Assuming it's an integer
data['Outstanding_Debt'] = data['Outstanding_Debt'].replace(r'[^0-9.]', '', regex=True).astype(float)

# Initialize StandardScaler
scaler = StandardScaler()

# List of numeric columns
numeric_cols = ['Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Outstanding_Debt']

# Apply scaling
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])


display(data)

,ID,Customer_ID,Month,Name,Age,SSN,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,...,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Occupation________,Credit_Mix_Good,Credit_Mix_Standard,Credit_Mix__
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.0,821-00-0265,19114.12,-0.744337,-0.120024,4,...,False,False,False,True,False,False,False,False,False,True
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23.0,821-00-0265,19114.12,-0.315849,-0.120024,4,...,False,False,False,True,False,False,False,True,False,False
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500.0,821-00-0265,19114.12,-0.315849,-0.120024,4,...,False,False,False,True,False,False,False,True,False,False
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23.0,821-00-0265,19114.12,-0.315849,-0.120024,4,...,False,False,False,True,False,False,False,True,False,False
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23.0,821-00-0265,19114.12,-0.744337,-0.120024,4,...,False,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25.0,078-73-5990,39628.99,-0.226137,-0.111506,6,...,True,False,False,False,False,False,False,False,False,True
99996,0x25fea,CUS_0x942c,May,Nicks,25.0,078-73-5990,39628.99,-0.226137,-0.111506,6,...,True,False,False,False,False,False,False,False,False,True
99997,0x25feb,CUS_0x942c,June,Nicks,25.0,078-73-5990,39628.99,-0.226137,-0.111506,6,...,True,False,False,False,False,False,False,True,False,False
99998,0x25fec,CUS_0x942c,July,Nicks,25.0,078-73-5990,39628.99,-0.226137,-0.111506,6,...,True,False,False,False,False,False,False,True,False,False


In [10]:
# remove the target column

X = data.drop('Credit_Score', axis=1)
y = data['Credit_Score']

In [11]:
# remove the target column
X = data.drop('Credit_Score', axis=1)
y = data['Credit_Score']

# Before splitting, ensure all X columns are numeric
# This is a crucial step to handle any remaining non-numeric values
for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Now proceed with the split
# split the traing datset to test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Drop the UnNecessery things
data = data.drop(columns=['Customer_ID', 'ID'])

In [12]:
# RandomForest

# import the lib

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate accuracy
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 77.96%
